In [1]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, dist
import geopy.distance

In [2]:
##### EXTRACT THE DATA #####

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
users_path = 'eua-dataset\\users\\'
U = pd.read_csv(users_path + 'users-test.csv')

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
servers_path = 'eua-dataset\\edge-servers\\'
S = pd.read_csv(servers_path + 'serverstest.csv')

In [3]:
##### COVERAGE OF EACH SERVER #####

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

U_si = []
dij = []


for i in range(len(S)):
  server_latitude = S.iloc[i]['LATITUDE']
  server_longitude = S.iloc[i]['LONGITUDE']
  temp = [] #MCF
  dist_temp = [] #VSBP
  for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']

    radius = 0.03 # in kilometer

    dist_geo = haversine(server_longitude, server_latitude, user_longitude, user_latitude)
    dist_temp.append(dist_geo)

    #print('Distance (km) : ', a)
    if dist_geo <= radius:
        print("User ", j+1, " is Inside the area of server ", i+1)
        temp.append(1)
        
    else:
        print("User", j+1, "NOT alocated in Server", i+1)
        temp.append(0)
        #print("User ", j+1, " is Outside the area of server ", i+1)
  print("---------------")
  U_si.append(temp)
  dij.append(dist_temp)

User  1  is Inside the area of server  1
User  2  is Inside the area of server  1
User  3  is Inside the area of server  1
User 4 NOT alocated in Server 1
User 5 NOT alocated in Server 1
User 6 NOT alocated in Server 1
User 7 NOT alocated in Server 1
User 8 NOT alocated in Server 1
---------------
User 1 NOT alocated in Server 2
User  2  is Inside the area of server  2
User  3  is Inside the area of server  2
User  4  is Inside the area of server  2
User  5  is Inside the area of server  2
User  6  is Inside the area of server  2
User 7 NOT alocated in Server 2
User 8 NOT alocated in Server 2
---------------
User  1  is Inside the area of server  3
User 2 NOT alocated in Server 3
User  3  is Inside the area of server  3
User 4 NOT alocated in Server 3
User 5 NOT alocated in Server 3
User  6  is Inside the area of server  3
User  7  is Inside the area of server  3
User  8  is Inside the area of server  3
---------------


In [4]:
#Criticality
du_Dt = [] 

#Position Danger
D = (-37.81952,144.95714099999998)

for j in range(len(U)):
  
  user_latitude = U.iloc[j]['Latitude']
  user_longitude = U.iloc[j]['Longitude']
  user_coordinates = (user_latitude,user_longitude)
  dist_geoDanger = geopy.distance.geodesic(user_coordinates, D).km
  du_Dt.append(dist_geoDanger)

# influence range of danger source D (radius)
D_rad = 100

# perception range of individual uj
U_per = np.full(len(U),40)

# R = [CPU, RAM, VRAM, bandwidth]
R = np.zeros((4), dtype=int)

# Criticality
CR = np.random.uniform(0.2,0.8,len(U))

# User u has memory requirements = w
W = []
NORM = []
for j in U:
  w = np.random.randint(1,4,4) # 4 random integers from the range of 1 to 3.
  W.append(w)
  
# Each server has memory capacity c
#C = np.full((len(D), len(S)), 10)
C = []
for i in S:
  c = np.random.randint(8,11,4) # 4 random integers from the range of 8 to 10.
  C.append(c)
 

In [7]:
###### CRITICALITY MODEL ###########

# scrj(t) = sej + emj (t) 

se_j =np.random.uniform(0.05,0.1, len(U))
lambd = 0.5
T_1 =0
T_2 = 61
T = T_2-T_1
#ti = list(range(T_1,T_2))
nej = np.random.uniform(0,1,len(U))


def em(ti):
    em_j = np.zeros(len(U))
    for j in range(len(U)):
        if du_Dt[j] < (D_rad+U_per[j]):
            eq = (ti-T_1)/T * (1-(du_Dt[j]/(D_rad+U_per[j])))*lambd* nej[j]
            em_j[j]=eq
        else:
            em_j[j]=0
    return em_j


In [ ]:
########## DECLARE THE MIP Solver ##########
solver = pywraplp.Solver.CreateSolver('SCIP')

In [ ]:
########## DEFINE THE VARIABLES ##########
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in index:
    for i in range(len(S)):
        x[i, j] = solver.BoolVar(f'c[{i}][{j}]')
        #print(x[i,j])

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in range(len(S)):
    y[i] = solver.BoolVar(f'c[{i}]')

print('Number of variables =', solver.NumVariables())

In [ ]:
########## DEFINE THE CONSTRAINTS ##########

#Proximity constraint
for j in index:
    for i in range(len(S)):
        if U_si[i][j] == 0:
            solver.Add(
             x[i, j]==0
            )

# For each server i
# the sum of memory requirements of the users allocated to server i
# does not exceed the memory capacity of i
for i in range(len(S)):
    for k in range(len(D)):
        solver.Add(
            solver.Sum([
                x[i, j] * W[j][k] for j in index
                #x[i, j] * W[j][k] for j in range(len(U))
            ]) <= C[i][k]*y[i]
        )

# Each user can be allocated just once
for j in index:
    solver.Add(
        solver.Sum([x[i, j] for i in range(len(S))]) <= 1
    )

# Constraint 3: x, y ≥ 0
# has already been included in the definition of the variables

print('Number of constraints =', solver.NumConstraints())

In [ ]:
########## DEFINE THE OBJECTIVE ##########

# Maximize the number of allocated users
objective = solver.Objective()
for j in index:
    for i in S:
        objective.SetCoefficient(x[i, j], 1)
objective.SetMaximization()

# Minimize the number of required edge servers
#objectiveMin = solver.Objective()

#for i in S:
 #   objectiveMin.SetCoefficient(y[i], 1)
#objectiveMin.SetMinimization()

In [ ]:
########## DEFINE THE OBJECTIVE ##########

# Maximize the number of allocated users
objective = solver.Objective()
for j in index:
    for i in range(len(S)):
        objective.SetCoefficient(x[i, j], 1)
objective.SetMaximization()

# Minimize the number of required edge servers
""" objectiveMin = solver.Objective()

for i in S:
    objectiveMin.SetCoefficient(y[i], 1)
objectiveMin.SetMinimization() """